In [87]:
# сортировка данных

import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')

melb_df.sort_values(by='Price') # индексы сохранились из исходной таблицы
melb_df.sort_values(by='Date', ascending=False) # сортировка по убыванию
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']] # сортировка по нескольким столбцам

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values( # фильтрация
    by=['Date', 'AreaRatio'], # сортировка
    ascending=[True, False], # порядок для каждого столбца
    ignore_index=True # замена старых индексов на новые
).loc[:, ['Date', 'AreaRatio']] 

# фильтрация х2, сортировка х2, замена индекса индекс, доступ к ячейке
melb_df[(melb_df['Type'] == 'townhouse') & (melb_df['Rooms'] > 2)].sort_values(
    by=['Rooms','MeanRoomsSquare'], ascending=[True,False], ignore_index=True).loc[18,'Price']


1300000.0

In [95]:
# группировка данных

melb_df.groupby(by='Type').mean() # группы стоят в качестве индекса
melb_df.groupby(by='Type', as_index=False).mean() # отображение групп в качестве отдельного столбца
melb_df.groupby('Type')['Price'].mean() # объект типа Series
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False) # сортировка после группировки

melb_df.groupby('MonthSale')['Price'].agg(['count', 'mean', 'max'] # несколько агрегирующих функций
).sort_values(by='count', ascending=False) # объект типа Dataframe
melb_df.groupby('MonthSale')['Price'].agg('describe') # все основные стат характеристики

melb_df.groupby('Regionname')['SellerG'].agg('nunique') # подсчет уникальных значений
melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set]) # выдача уникальных значений в виде множества

# фильтрация х2, группировка, сумма, сортировка х2
melb_df[(melb_df['Date'] >= '2017-05-01') & (melb_df['Date'] <= '2017-09-01')].groupby(by='SellerG')['Price'].sum().sort_values()

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{RT, Love, Gary, Fletchers, McGrath, Barry, Ka..."
Eastern Victoria,11,"{HAR, Barry, Harcourts, C21, other, Ray, O'Bri..."
Northern Metropolitan,40,"{RT, Greg, Love, Raine, Rendina, McDonald, Ale..."
Northern Victoria,11,"{HAR, Barry, Buckingham, Raine, McDonald, YPA,..."
South-Eastern Metropolitan,25,"{Greg, Gary, O'Brien, Fletchers, Eview, McGrat..."
Southern Metropolitan,38,"{RT, Greg, Raine, Rendina, Gary, Nick, O'Brien..."
Western Metropolitan,34,"{RT, Greg, Douglas, McDonald, Rendina, Raine, ..."
Western Victoria,6,"{HAR, Raine, YPA, other, Ray, hockingstuart}"


In [156]:
# сводные таблицы

melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack() # группировка по нескольким признакам (первый - строки, второй — столбцы)

pivot = melb_df.pivot_table( # создание сводной таблицы
    values='Landsize',
    index='Regionname', # имя или список строк
    columns='Type', # имя или список столбцов
    aggfunc=['median', 'mean'], # имя или список агрегирующих функций (по умолчанию — mean)
    fill_value=0 # значение, которым необходимо заполнить пропуски
)
pivot.columns # объекта типа MultiIndex, хранит все возможные варианты обращения к столбцам
pivot['mean']['unit'] # последовательное обращение к столбцам
filtered_pivot = pivot[pivot['mean']['house'] < pivot['median']['house']] # фильтрация внутри сводной
list(filtered_pivot.index) # индексы сводной

['Southern Metropolitan', 'Western Metropolitan']

In [235]:
# объединение таблиц

import pandas as pd

ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
movies = pd.read_csv('data/movies.csv')
dates = pd.read_csv('data/dates.csv')

ratings = pd.concat( # присоединение таблицы снизу
    [ratings1, ratings2],
    ignore_index=True # назначение нового сквозного индекса
)

ratings = ratings.drop_duplicates(ignore_index=True) # удаление дублирующихся строк и переназначение индекса

ratings_dates = pd.concat([ratings, dates], axis=1) # присоединение таблицы справа

joined = ratings_dates.join(
    movies.set_index('movieId'), # ключ в правой таблице
    on='movieId', # ключ в основной таблице
    #rsuffix='_right', # если название столбцов дублируются, к правым названиям присоединяется суффикс
    how='left'
)

merged = ratings_dates.merge(
    movies,
    #left_on = 'movieID' # если названия ключевых столбцов не совпадают
    #right_on = 'movieID'
    on='movieId',
    how='left'
)

merge_ratings = ratings1.merge(ratings2, how='outer') # в отличие от метода concat пропадают все дубликаты

In [34]:
# задание: данные по фильмам 

import pandas as pd

ratings_movies = pd.read_csv('data/ratings_movies.csv')
ratings_movies.head()

import re #библиотека для регулярных выражений

from numpy import int64 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies = ratings_movies.drop(['Unnamed: 0'], axis=1)

# У скольких фильмов не указан год их выпуска?
ratings_movies['year_release'].isnull().sum()

#Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
ratings_movies[ratings_movies['year_release'] == 2010].groupby('genres').mean().sort_values(by='rating')

# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
ratings_movies.groupby(['userId'])['genres'].value_counts().groupby('userId').value_counts().sort_values(ascending=False)

# Какой пользователь выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая?
ratings_movies.groupby(['userId'])['rating'].agg(['count','mean']).sort_values(by=['count','mean'], ascending=[True,False])

# Какие жанры в 2018 году имели наибольший средний рейтинг, и при этом число выставленных им оценок больше 10?
ratings_movies[ratings_movies['year_release'] == 2018].groupby('genres')['rating'].agg(['count','mean']).sort_values(by=['count','mean'], ascending=[False,False])

ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year

,userId,movieId,rating,date,title,genres,year_release,year_rating
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,2000
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0,2000
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0,2000
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0,2000
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0,2000
...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller,2017.0,2017
100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller,2017.0,2017
100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror,2017.0,2017
100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi,2017.0,2017


In [180]:
# задание: интернет заказы 

import pandas as pd

orders = pd.read_csv('data/orders.csv',sep=';')
products = pd.read_csv('data/products.csv',sep=';')

orders.head(5)
products.head(5)

orders_products = orders.join(
    products.set_index('Product_ID'), # ключ в правой таблице
    on='ID товара', # ключ в основной таблице
    how='left'
)

# Какой идентификатор имеет заказ, для которого не оказалось информации о товаре?
orders_products[orders_products['Name'].isnull() == True][['Order ID','Name']]

# На какой товар была произведена отмена?
orders_products[orders_products['Отменен'] == 'Да'][['Order ID','Name']]

# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.

orders_products['Revenue'] = orders_products[orders_products['Оплачен'] == 'Да']['Price']*orders_products[orders_products['Оплачен'] == 'Да']['Количество']
orders_products.groupby('ID Покупателя')['Revenue'].max()

ID Покупателя
1        NaN
5     7999.0
7     8999.0
8      747.0
9        NaN
10       NaN
Name: Revenue, dtype: float64